# Trading strategy implementation with Zipline

This is the script for implementing our trading algorithm with Zipline Librairy and Quantopian platform. 
 Credits:
 http://www.zipline.io/index.html
 https://www.quantopian.com/

In [ ]:
from zipline.api import (symbol,sid, order, record, fetch_csv,set_benchmark,schedule_function,
        date_rules,time_rules,order_target_percent,order_target,get_datetime,set_commission,commission )
from zipline import run_algorithm
import numpy as np
import datetime
import matplotlib.pyplot as plt
import pytz
from datetime import datetime
import pandas as pd


def drop_na(df):
    return df.fillna(0)

def backtest(ticker,delta_t1,delta_t2):
    def initialize(context):
        # This is the search query we are using, this is tied to the csv file.
        context.query = str(ticker)  #column name for each stock in our universe
        # User fetcher to get data. I uploaded this csv file manually.
        # Note that this data is daily averages.
        fetch_csv(csv_file_path ,  #
                  date_format='%Y-%m-%d',
                  pre_func =drop_na,
                  symbol='wiki',
                  skiprows=0
        )
        
        # set_commission(commission.PerTrade(cost=0.00 )) Optionnal: Set the transaction cost
        
         #Define our securities, global variables, and schedule when to trade.
        context.security = symbol('SPY')
        #set_benchmark(symbol(ticker)) # S&P5000 # stock refering to context.query 
        
        #we trade every day at market opening time
        schedule_function(rebalance, 
                          date_rule=date_rules.every_day(), #day or week
                          time_rule=time_rules.market_open())

        context.past_queries_t1 = []
        context.past_queries_t2 = []

    def rebalance(context, data):
        c = context
        dt=get_datetime()
        # Extract weekly average of search query.
        indicator = data.current('wiki',c.query);  #c.query=nom de la colonne dans data
        
        '''Compute the moving average t1'''
        # Track our fetched values in a context variable to build up history.
        context.past_queries_t1.append(indicator)

        if len(context.past_queries_t1) > delta_t1:
            del context.past_queries_t1[0]

        if len(context.past_queries_t1) == delta_t1:
            # Compute average over weeks in range
            mean_indicator_t1 = np.mean(context.past_queries_t1)
            
        ''' Compute the moving average t2''' 
        # Track our fetched values in a context variable to build up history.
        context.past_queries_t2.append(indicator)

        if len(context.past_queries_t2) > delta_t2:
            del context.past_queries_t2[0]

        if len(context.past_queries_t2) == delta_t2:
            # Compute average over weeks in range
            mean_indicator_t2 = np.mean(context.past_queries_t2)
            
            # Long or short depending on whether moving averages level.
            # went down or up, respectively.
            if np.isnan(mean_indicator_t2):
                order_target_percent(c.security, 0)
  
            elif mean_indicator_t1>mean_indicator_t2 and data.can_trade(c.security):
                order_target_percent(c.security, -1.0)

            elif mean_indicator_t1<mean_indicator_t2  and data.can_trade(c.security):
                order_target_percent(c.security, 1.0)
            else:
                order_target_percent(c.security, 0)
                
    # Set up the stuff for running the trading simulation
    start = datetime(2008, 1, 1, 0, 0, 0, 0, pytz.utc)
    end = datetime(2016, 12, 31, 0, 0, 0, 0, pytz.utc)
    base_capital = 10000

    # run the trading algorithm and save the results in perf
    perf = run_algorithm(start, end, initialize, base_capital,bundle = 'quantopian-quandl')
    return perf

### Launch series of backtest

In [ ]:

file=pd.read_csv('',index_col=0) #Search query csv file path
period_return=pd.DataFrame()
temp=pd.DataFrame()

list_delta_t1=[,] # List of short maturity moving av in days
list_delta_t2=[,,] #List of long maturity moving av in days

#for each keywords in sample 
for column in file.columns:
    
#calculate the return over the total period for various maturity of moving average    
    for delta_t1 in list_delta_t1:
        for delta_t2 in list_delta_t2:
            if delta_t1<delta_t2:
                try:
                    # for each tuple (delta_t1,delta_t2), compute the end of period return
                    temp[(delta_t1,delta_t2)]=[return_over_period(column,delta_t1,delta_t2)]   #column name is tuple with (delta_t1,delta_t2)
                    temp.index=[str(column)]
                    print(temp.T.tail())#index is the stock name
                except:
                    pass
                
    period_return=pd.concat([period_return,temp.T],axis=1,join='outer')
    
period_return.to_excel('## Output')                